In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 22479188
paper_name = 'albulescu_pleiss_2012' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [29]:
original_data = pd.read_excel('raw_data/hits.xlsx', sheet_name='Sheet1')

In [30]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 422 x 8


In [31]:
original_data.head()

,NAME,Strain,U3,Tef5,Rpl31b,Tub3,Ubc13,Max score
0,YKL149C,dbr1Δ,42.225520,NaN,11.827286,NaN,NaN,42.225520
1,YLR147C,smd3-ts,24.461928,4.273349,3.504766,2.785785,6.843757,24.461928
2,YIR005W,ist3Δ,24.426979,5.007953,5.460240,2.847594,8.792734,24.426979
3,YGR278W,cwc22-ts,22.366594,3.890875,4.282662,2.207389,6.240979,22.366594
4,YHR165C,prp8-1,21.087150,4.140007,4.171478,4.493496,6.474045,21.087150


In [32]:
original_data['orf'] = original_data['NAME'].astype(str)

In [33]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [34]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [35]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [NAME, Strain, U3, Tef5, Rpl31b, Tub3, Ubc13, Max score, orf]
Index: []


In [36]:
original_data.set_index('orf', inplace=True)

In [37]:
original_data = original_data[['U3','Tef5','Rpl31b','Tub3','Ubc13']].copy()

In [38]:
original_data = original_data.groupby(original_data.index).mean()

In [39]:
original_data.shape

(418, 5)

In [41]:
original_data[original_data.isnull()] = 1

In [42]:
original_data.head()

,U3,Tef5,Rpl31b,Tub3,Ubc13
orf,,,,,
YAL016C-B,1.0,2.077287,2.449803,1.000000,1.000000
YAL016W,1.0,1.000000,1.000000,4.117393,1.000000
YAL021C,1.0,1.000000,2.840629,1.000000,1.000000
YAL036C,1.0,1.000000,1.000000,1.000000,2.128698
YAL044W-A,1.0,1.000000,3.400784,1.000000,1.000000


# Load & process tested strains

In [43]:
tested = pd.read_csv('raw_data/no_hits.txt', sep='\t')

In [44]:
tested.head()

,Systematic name
0,YAL002W
1,YAL004W
2,YAL005C
3,YAL007C
4,YAL008W


In [45]:
tested['orf'] = tested['Systematic name'].astype(str)

In [46]:
tested['orf'] = clean_orf(tested['orf'])

In [47]:
tested.loc[tested['orf']=='YLR287-A','orf'] = 'YLR287C-A'
tested.loc[tested['orf']=='YCRO54C','orf'] = 'YCR054C'

In [48]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [49]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [Systematic name, orf]
Index: []


In [50]:
tested = tested.loc[t,:]

In [51]:
tested_orfs = tested['orf'].unique()

In [52]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [53]:
original_data = original_data.reindex(index=tested_orfs, fill_value=1)

# Prepare the final dataset

In [54]:
data = original_data.copy()

In [55]:
dataset_ids = [16311, 16315, 16312, 16314, 16313]
datasets = datasets.reindex(index=dataset_ids)

In [56]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [57]:
data.head()

dataset_id,16311,16315,16312,16314,16313
data_type,value,value,value,value,value
orf,,,,,
YAL002W,1.0,1.0,1.0,1.0,1.0
YAL004W,1.0,1.0,1.0,1.0,1.0
YAL005C,1.0,1.0,1.0,1.0,1.0
YAL007C,1.0,1.0,1.0,1.0,1.0
YAL008W,1.0,1.0,1.0,1.0,1.0


## Subset to the genes currently in SGD

In [58]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [59]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16311,16315,16312,16314,16313
,data_type,value,value,value,value,value
gene_id,orf,,,,,
2,YAL002W,1.0,1.0,1.0,1.0,1.0
1863,YAL004W,1.0,1.0,1.0,1.0,1.0
4,YAL005C,1.0,1.0,1.0,1.0,1.0
5,YAL007C,1.0,1.0,1.0,1.0,1.0
6,YAL008W,1.0,1.0,1.0,1.0,1.0


# Normalize

In [60]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [61]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [62]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16311,16315,16312,16314,16313,16311,16315,16312,16314,16313
,data_type,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,
2,YAL002W,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1863,YAL004W,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,YAL005C,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
5,YAL007C,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
6,YAL008W,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


# Print out

In [63]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [64]:
from IO.save_data_to_db3 import *

In [65]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/5 [00:00<?, ?it/s]

Deleting all datasets for PMID 22479188...
Inserting the new data...


100%|██████████| 5/5 [00:41<00:00,  8.23s/it]

Updating the data_modified_on field...
